In [4]:
import numpy as np
import plotly.express as px
import pandas as pd

data = pd.read_csv('/home/becode/Desktop/becode_projects/Real-Estate-Price-Prediction/Data_Analysis/data/dataformodelling.csv')
print(data.shape)
data.isnull().sum()

(17940, 33)


Unnamed: 0                                                   0
classified_id                                                0
classified_type                                              0
classified_subtype                                           0
classified_price                                             0
classified_transactionType                                   0
classified_zip                                               0
classified_visualisationOption                               0
classified_kitchen_type                                      0
classified_building_constructionYear                      6960
classified_building_condition                                0
classified_energy_heatingType                             5344
classified_certificates_primaryEnergyConsumptionLevel     4373
classified_bedroom_count                                  1357
classified_land_surface                                      0
classified_atticExists                                 

In [5]:
data1 = data[data.columns[data.isnull().sum() < 10000]]
data1.head()

,Unnamed: 0,classified_id,classified_type,classified_subtype,classified_price,classified_transactionType,classified_zip,classified_visualisationOption,classified_kitchen_type,classified_building_constructionYear,...,classified_outdoor_terrace_exists,new_price,classified_region,bedroom_count,types,encoded_bedrooms,encoded_region,encoded_kitchen,encoded_heating,avg_sal_zip_code
0,0,10146915,apartment,apartment,339000,forsale,9890,xl,hyperequipped,2021.0,...,True,0,Oost Vlaanderen,2,1,2,9,4,1,18847.33
1,1,10156372,apartment,apartment,395000,forsale,1140,xl,hyperequipped,2022.0,...,True,0,Brussels,3,1,3,3,4,1,21079.33
2,3,10157770,apartment,apartment,249000,forsale,1800,xl,installed,2010.0,...,True,0,Vlaams Brabant,2,1,2,10,2,1,18847.33
3,4,10157545,apartment,apartment,325000,forsale,8380,xl,hyperequipped,1967.0,...,True,0,West Vlaanderen,2,1,2,11,4,1,18847.33
4,5,10157660,apartment,duplex,369000,forsale,9000,xl,hyperequipped,2007.0,...,True,0,Oost Vlaanderen,2,1,2,9,4,1,18847.33


In [74]:
print(data1['classified_price'].corr(data1['classified_land_surface']))
print(data1['classified_price'].corr(data1['avg_sal_zip_code']))
print(data1['classified_price'].corr(data1['types']))
print(data1['classified_price'].corr(data1['encoded_bedrooms']))
print(data1['classified_price'].corr(data1['encoded_kitchen']))
print(data1['classified_price'].corr(data1['encoded_heating']))


0.21810663058252397
0.2580686203344817
0.14402844425968248
0.18853909344443606
0.18056447321849967
-0.051182886724236605


In [10]:
fig = px.scatter(data1, x = 'classified_land_surface', y = 'classified_price')
fig.show()

fig = px.scatter(data1, x = 'avg_sal_zip_code', y = 'classified_price',color = 'classified_type')
fig.show()

In [12]:
data2 = data1[['classified_price','classified_land_surface','avg_sal_zip_code']].copy()
data2.head()

,classified_price,classified_land_surface,avg_sal_zip_code
0,339000,116.896552,18847.33
1,395000,136.206897,21079.33
2,249000,85.862069,18847.33
3,325000,112.068966,18847.33
4,369000,127.241379,18847.33


In [47]:
y = np.array(data2['classified_price'])
X = np.array(data2.drop('classified_price',axis = 1))
print(X.shape)
print(y.shape)

(17940,)
(17940,)


In [14]:
# spliting into train and test set
# Normalising the land surface area

y = np.array(data1['classified_price'])
X = np.array(data1['classified_land_surface'])
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X_train,X_test,y_train,y_test = train_test_split(X, y, random_state = 41, test_size = 0.2)

scaler = StandardScaler()
print(scaler.fit(X_train.reshape(-1,1)))
print(scaler.mean_)
X_train_scaled = scaler.transform(X_train.reshape(-1,1))
X_test_scaled = scaler.transform(X_test.reshape(-1,1))



StandardScaler()
[456.96494721]


In [49]:
# training the model using polynomialregression degree 10 gives the highes score

degree = 10

from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures

poly_model = make_pipeline(PolynomialFeatures(degree), LinearRegression())

poly_model.fit(X_train_scaled, y_train)
a = poly_model.score(X_train_scaled, y_train)
print(a)

0.15832106367647636


In [50]:
y_pred = poly_model.predict(X_test_scaled)
b = poly_model.score(X_test_scaled,y_test)
print(b)

0.16099178262354574


In [21]:
# training the model using multiple linear regression 

#splitting into features and target
y2 = np.array(data2['classified_price'])
X2 = np.array(data2.drop('classified_price',axis = 1))

#splitting into train and test sets
from sklearn.model_selection import train_test_split
X2_train,X2_test,y2_train,y2_test = train_test_split(X2, y2, random_state = 41, test_size = 0.2)

#normalisation
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
print(scaler.fit(X2_train))
print(scaler.mean_)
X2_train_scaled = scaler.transform(X2_train)
X2_test_scaled = scaler.transform(X2_test)

#multiple linear regression
from sklearn.linear_model import LinearRegression
model = LinearRegression().fit(X2_train_scaled,y2_train)
r_sq = model.score(X2_train_scaled,y2_train)
print(r_sq)

y_pred = model.predict(X2_test)
r_sq_test = model.score(X2_test_scaled,y2_test)
print(r_sq_test)

StandardScaler()
[  456.96494721 18989.24838528]
0.11080599058390983
0.1156664338153226


In [62]:
degree = 8

from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures

poly_model2 = make_pipeline(PolynomialFeatures(degree), LinearRegression())

poly_model2.fit(X2_train_scaled, y2_train)
a = poly_model2.score(X2_train_scaled, y2_train)
print(a)
y_pred2 = poly_model2.predict(X2_test_scaled)
b = poly_model2.score(X2_test_scaled,y2_test)
print(b)

0.22239573900276732
0.1905263112228851


In [69]:
data3 = data1[['classified_price','classified_land_surface','types',
               'encoded_bedrooms','encoded_kitchen',
               'avg_sal_zip_code']].copy()
data3.head()

#splitting into features and target
y3 = np.array(data3['classified_price'])
X3 = np.array(data3.drop('classified_price',axis = 1))

#splitting into train and test sets
from sklearn.model_selection import train_test_split
X3_train,X3_test,y3_train,y3_test = train_test_split(X3, y3, random_state = 41, test_size = 0.2)

#normalisation
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
print(scaler.fit(X3_train))
print(scaler.mean_)
X3_train_scaled = scaler.transform(X3_train)
X3_test_scaled = scaler.transform(X3_test)

#polynomial Regression
degree_range = range (1,7)
score_list = []

from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures

for degree in degree_range:
    poly_model3 = make_pipeline(PolynomialFeatures(degree), LinearRegression())
    poly_model3.fit(X3_train_scaled, y3_train)
    #a = poly_model3.score(X3_train_scaled, y3_train)
    y_pred3 = poly_model3.predict(X3_test_scaled)
    score_list.append(poly_model3.score(X3_test_scaled,y3_test))
    
#plot the scores vs degree of the polynomial expression
fig = px.line(x = degree_range, y = score_list)
fig.show()
    
#training the polynomial regression model with the degree with best score
final_degree = 4
poly_model3 = make_pipeline(PolynomialFeatures(final_degree), LinearRegression())
poly_model3.fit(X3_train_scaled, y3_train)
a = poly_model3.score(X3_train_scaled, y3_train)
print(a)
y_pred3 = poly_model3.predict(X3_test_scaled)
b = poly_model3.score(X3_test_scaled,y3_test)
print(b)



StandardScaler()
[4.56964947e+02 1.48369565e+00 3.25215998e+00 2.76226310e+00
 1.89892484e+04]


0.695316638270332
0.6774751872848428


In [76]:
data1['classified_bedroom_count'].mode()
data1['classified_bedroom_count'] = data1['classified_bedroom_count'].fillna(data1['classified_bedroom_count'].mode()[0])
data1['classified_bedroom_count'].isnull().sum()
print(data1['classified_price'].corr(data1['classified_bedroom_count']))


0.3942610284785918


/home/becode/.local/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [113]:
data4 = data1[['classified_price','classified_land_surface','types',
               'classified_bedroom_count','encoded_kitchen',
               'avg_sal_zip_code']].copy()
data4.head()

#splitting into features and target
y4 = np.array(data4['classified_price'])
X4 = np.array(data4.drop('classified_price',axis = 1))

#splitting into train and test sets
from sklearn.model_selection import train_test_split
X4_train,X4_test,y4_train,y4_test = train_test_split(X4, y4, random_state = 41, test_size = 0.2)

#normalisation
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
print(scaler.fit(X4_train))
print(scaler.mean_)
X4_train_scaled = scaler.transform(X4_train)
X4_test_scaled = scaler.transform(X4_test)

#polynomial Regression
degree_range = range (1,7)
score_list = []

from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures

for degree in degree_range:
    poly_model4 = make_pipeline(PolynomialFeatures(degree), LinearRegression())
    poly_model4.fit(X4_train_scaled, y4_train)
    #a = poly_model4.score(X4_train_scaled, y4_train)
    y_pred4 = poly_model4.predict(X4_test_scaled)
    score_list.append(poly_model4.score(X4_test_scaled,y4_test))
    
#plot the scores vs degree of the polynomial expression
fig = px.line(x = degree_range, y = score_list)
fig.show()


StandardScaler()
[4.56964947e+02 1.48369565e+00 2.73216276e+00 2.76226310e+00
 1.89892484e+04]


In [153]:
    
#training the polynomial regression model with the degree with best score
final_degree = 5
poly_model4 = make_pipeline(PolynomialFeatures(final_degree), LinearRegression())
poly_model4.fit(X4_train_scaled, y4_train)
a = poly_model4.score(X4_train_scaled, y4_train)
print(a)
y_pred4 = poly_model4.predict(X4_test_scaled)
b = poly_model4.score(X4_test_scaled,y4_test)
print(b)

#plot the test and predicted 
import plotly.graph_objects as go
import plotly.express as px
fig = px.scatter(x=y4_test, y=y_pred, labels={'x': 'actual', 'y': 'prediction'})
fig.add_shape(
    type="line", line=dict(dash='dash'),
    x0=y_pred.min(), y0=y_pred.min(),
    x1=y_pred.max(), y1=y_pred.max()
)
fig.show()

0.6991209277880458
0.667825285401641


In [95]:
data1.head()
from sklearn.preprocessing import OneHotEncoder

#creating instance of one-hot-encoder
encoder = OneHotEncoder(handle_unknown='ignore')

#perform one-hot encoding on 'team' column 
encoder_types = pd.DataFrame(encoder.fit_transform(data1[['classified_kitchen_type']]).toarray())

#merge one-hot encoded columns back with original DataFrame
final_data1 = data1.join(encoder_types)

#view final df
final_data1.head()


,Unnamed: 0,classified_id,classified_type,classified_subtype,classified_price,classified_transactionType,classified_zip,classified_visualisationOption,classified_kitchen_type,classified_building_constructionYear,...,types,encoded_bedrooms,encoded_region,encoded_kitchen,encoded_heating,avg_sal_zip_code,0,1,2,3
0,0,10146915,apartment,apartment,339000,forsale,9890,xl,hyperequipped,2021.0,...,1,2,9,4,1,18847.33,1.0,0.0,0.0,0.0
1,1,10156372,apartment,apartment,395000,forsale,1140,xl,hyperequipped,2022.0,...,1,3,3,4,1,21079.33,1.0,0.0,0.0,0.0
2,3,10157770,apartment,apartment,249000,forsale,1800,xl,installed,2010.0,...,1,2,10,2,1,18847.33,0.0,1.0,0.0,0.0
3,4,10157545,apartment,apartment,325000,forsale,8380,xl,hyperequipped,1967.0,...,1,2,11,4,1,18847.33,1.0,0.0,0.0,0.0
4,5,10157660,apartment,duplex,369000,forsale,9000,xl,hyperequipped,2007.0,...,1,2,9,4,1,18847.33,1.0,0.0,0.0,0.0


In [99]:
print(final_data1['classified_price'].corr(final_data1[3]))

-0.14064191006710333


In [109]:
#trying to add additional feature (but idea dropped since the correlation is not good)
data1['classified_certificates_primaryEnergyConsumptionLevel'].head()
data1['classified_certificates_primaryEnergyConsumptionLevel'].isnull().sum()
data1['classified_certificates_primaryEnergyConsumptionLevel'].median()
data1['classified_certificates_primaryEnergyConsumptionLevel'] = data1['classified_certificates_primaryEnergyConsumptionLevel'].fillna(data1['classified_certificates_primaryEnergyConsumptionLevel'].median())
data1['classified_certificates_primaryEnergyConsumptionLevel'].isnull().sum()
print(data1['classified_price'].corr(data1['classified_certificates_primaryEnergyConsumptionLevel']))


0.008898995292937237


/home/becode/.local/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [112]:
#model after One Hot Encoding of kitchen type

data5 = final_data1[['classified_price','classified_land_surface','types',
               'classified_bedroom_count','avg_sal_zip_code',0,1,2,3]].copy()
data5.head()

#splitting into features and target
y5 = np.array(data5['classified_price'])
X5 = np.array(data5.drop('classified_price',axis = 1))

#splitting into train and test sets
from sklearn.model_selection import train_test_split
X5_train,X5_test,y5_train,y5_test = train_test_split(X5, y5, random_state = 41, test_size = 0.2)

#normalisation
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
print(scaler.fit(X5_train))
print(scaler.mean_)
X5_train_scaled = scaler.transform(X5_train)
X5_test_scaled = scaler.transform(X5_test)

#polynomial Regression
degree_range = range (1,5)
score_list = []

from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures

for degree in degree_range:
    poly_model5 = make_pipeline(PolynomialFeatures(degree), LinearRegression())
    poly_model5.fit(X5_train_scaled, y5_train)
    #a = poly_model5.score(X5_train_scaled, y5_train)
    y_pred5 = poly_model5.predict(X5_test_scaled)
    score_list.append(poly_model5.score(X5_test_scaled,y5_test))
    
#plot the scores vs degree of the polynomial expression
fig = px.line(x = degree_range, y = score_list)
fig.show()


StandardScaler()
[4.56964947e+02 1.48369565e+00 2.73216276e+00 1.89892484e+04
 1.96976031e-01 3.54306020e-01 4.02034560e-02 4.08514493e-01]


In [131]:
# Random Forest Regressor Model

data6 = data1[['classified_price','classified_land_surface','types',
               'classified_bedroom_count','encoded_kitchen',
               'avg_sal_zip_code']].copy()
data6.head()

#splitting into features and target
y6 = np.array(data6['classified_price'])
X6 = np.array(data6.drop('classified_price',axis = 1))

#splitting into train and test sets
from sklearn.model_selection import train_test_split
X6_train,X6_test,y6_train,y6_test = train_test_split(X6, y6, random_state = 41, test_size = 0.2)

#normalisation
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X6_train)
print(scaler.mean_)
X6_train_scaled = scaler.transform(X6_train)
X6_test_scaled = scaler.transform(X6_test)


#Random Forest Regression
from sklearn.ensemble import RandomForestRegressor
clf = RandomForestRegressor(n_estimators=300)
clf.fit(X6_train_scaled, y6_train)
y_pred = clf.predict(X6_test_scaled)

#Metric Evaluation
c = clf.score(X6_test_scaled,y6_test)
print(c)

from sklearn.metrics import mean_squared_error
mse = mean_squared_error(y6_test, y_pred)
print(mse)
print(mse**.5)

[4.56964947e+02 1.48369565e+00 2.73216276e+00 2.76226310e+00
 1.89892484e+04]
0.7034261487009406
9331946443.829275
96602.0002061514


In [150]:
#plotting the y_test and y_pred
fig = px.scatter(x =y_pred, y=y6_test)
fig.show()

In [154]:
#model evaluation

from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

print("MAE",mean_absolute_error(y6_test,y_pred))
print("MSE",mean_squared_error(y6_test,y_pred))
print("RMSE",np.sqrt(mean_squared_error(y6_test,y_pred)))
print("RLMSE",np.log(np.sqrt(mean_squared_error(y6_test,y_pred))))
r2 = r2_score(y6_test,y_pred)
print("r2",r2)
n=3588
k=5
adj_r2_score = 1 - ((1-r2)*(n-1)/(n-k-1))
print("adj_r2_score",adj_r2_score)



MAE 50094.76409530379
MSE 9331946443.829275
RMSE 96602.0002061514
RLMSE 11.47835472605406
r2 0.7034261487009406
adj_r2_score 0.7030121706840519


In [1]:
#plotting the y_test and y_pred
fig = px.scatter(x =y_pred, y=y6_test)
fig.show()

NameError: name 'px' is not defined